In [21]:
from langfuse.callback import CallbackHandler
import keys 
import os
os.environ["LANGFUSE_HOST"]= "http://localhost:3000"
os.environ["LANGFUSE_SECRET_KEY"]= keys.langfuse_secret_key
os.environ["LANGFUSE_PUBLIC_KEY"]= keys.langfuse_public_key
os.environ["GOOGLE_API_KEY"]= keys.gemini_api_key
langfuse_handler = CallbackHandler()
langfuse_handler.auth_check()

#chain.invoke({"topic": "humans"}, config={"callbacks": [langfuse_handler]})

True

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [25]:
loader= PyPDFLoader("C:/Users/HP/Downloads/sample.pdf")
docs= loader.load_and_split()

In [27]:
text_splitter= RecursiveCharacterTextSplitter(
    chunk_size= 200,
    chunk_overlap= 20
)

documents= text_splitter.split_documents(docs)

In [29]:
len(documents)

36

In [31]:
embeddings= GoogleGenerativeAIEmbeddings(model= "models/embedding-001")

In [35]:
embedded_docs= embeddings.embed_query(documents[0].page_content)
len(embedded_docs)

768

In [39]:
vectorstore= FAISS.from_documents(documents, embeddings)
retriever= vectorstore.as_retriever()

In [41]:
retriever.invoke("how to process a pdf file?")

[Document(metadata={'source': 'C:/Users/HP/Downloads/sample.pdf', 'page': 4}, page_content='2.run“pdflatexfile.tex ”onthemainﬁleofthedocumentthreetimes\n(three—to preparevalidtable ofcontents);\n3.to see or print the result use acroread (unfortunately some versions of'),
 Document(metadata={'source': 'C:/Users/HP/Downloads/sample.pdf', 'page': 2}, page_content='Contents\n1 Template 5\n1.1 How tocompilea .texﬁleto a.pdfﬁle. . . . . . . . . . . . . 5\n1.1.1 Tools . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5'),
 Document(metadata={'source': 'C:/Users/HP/Downloads/sample.pdf', 'page': 6}, page_content='When producing.pdfﬁles you should usepdflatex , which produces di-\nrectly.pdfﬁles out of.texsources. Note that in the .texﬁle you may need\nto usesomePDFspeciﬁcpackages.'),
 Document(metadata={'source': 'C:/Users/HP/Downloads/sample.pdf', 'page': 5}, page_content='5.run“fax send phone-numberfile.ps ”asroottosendafax,or—ifyou\nknowhowtodothis—modifythe faxscripttobeabletofax .pd

In [81]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool= create_retriever_tool(
    retriever= retriever,
    name= "retriever_tool",
    description= '''For any information related to types of file and formattigs, use this tool.
    Useful for retrieving information related to the PDFs, Text files, Latex etc..''' 
)

retriever_tool.dict()

{'name': 'retriever_tool',
 'description': 'For any information related to types of file and formattigs, use this tool.\n    Useful for retrieving information related to the PDFs, Text files, Latex etc..',
 'args_schema': langchain_core.tools.RetrieverInput,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content',
 'func': functools.partial(<function _get_relevant_documents at 0x0000027219CCD260>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002721C7B3450>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'),
 'coroutine': functools.partial(<function _aget_relevant_documents at 0x0000027219CCD800>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_

In [47]:
search_tool= DuckDuckGoSearchRun()
search_tool.dict()

{'name': 'duckduckgo_search',
 'description': 'A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.',
 'args_schema': langchain_community.tools.ddg_search.tool.DDGInput,
 'return_direct': False,
 'verbose': False,
 'tags': None,
 'metadata': None,
 'handle_tool_error': False,
 'handle_validation_error': False,
 'response_format': 'content',
 'api_wrapper': {'region': 'wt-wt',
  'safesearch': 'moderate',
  'time': 'y',
  'max_results': 5,
  'backend': 'api',
  'source': 'text'}}

In [49]:
tools= [search_tool, retriever_tool]

In [53]:
from langchain_google_genai import GoogleGenerativeAI
llm= GoogleGenerativeAI(
    model= "gemini-1.5-pro",
    geoogle_api_key= os.environ["GOOGLE_API_KEY"]
)

In [55]:
llm

GoogleGenerativeAI(model='gemini-1.5-pro', client=genai.GenerativeModel(
    model_name='models/gemini-1.5-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))

In [57]:
from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

In [59]:
prompt= hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [63]:
agent= create_react_agent(tools= tools, llm= llm, prompt= prompt)

In [65]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\nretriever_tool(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002721C7B3450>), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator: 'str' = '\\n\\n', callbacks: 'Callbacks' = None) -> 'str' - For any information related to types of file and formattigs, use this tool.\n    Useful for retrieving information related to the PDFs, Text files, Latex etc..", 'tool_names': 'duckduckgo_search, retriever_tool'}, meta

In [67]:
agent_executor= AgentExecutor(agent= agent, tools= tools, verbose= True)

In [69]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'tools': "duckduckgo_search - A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.\nretriever_tool(query: 'str', *, retriever: 'BaseRetriever' = VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002721C7B3450>), document_prompt: 'BasePromptTemplate' = PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator: 'str' = '\\n\\n', callbacks: 'Callbacks' = None) -> 'str' - For any information related to types of file and formattigs, use this tool.\n    Useful for retrieving information related to the PDFs, Text files, Latex etc..",

In [77]:
agent_executor.invoke(
    {"input": "What is the hype behing strawberry in the context of AI?"},
    {"config": {"callbacks": [langfuse_handler]}}
)



> Entering new AgentExecutor chain...
Question: What is the hype behing strawberry in the context of AI?
Thought: I need more information about the relationship between "strawberry" and "AI". I'll use a search engine to find potential connections. 
Action: duckduckgo_search
Action Input: "strawberry" AI Immediately, the AI afficionados took to the comment section to decode the post. One user counted the strawberries in the picture, which totaled five, and correlated it to a possible release of ... July 15, 20247:37 AM PDTUpdated 22 days ago. July 12 - ChatGPT maker OpenAI is working on a novel approach to its artificial intelligence models in a project code-named "Strawberry," according ... A cryptic post published on X yesterday by OpenAI Sam Altman has the AI community buzzing with speculation about a mysterious new AI project. "I love summer in the garden," Altman wrote, above ... The documents describing the project and the source who spoke to Reuters suggests Strawberry is build

{'input': 'What is the hype behing strawberry in the context of AI?',
 'output': 'The hype surrounding "strawberry" in the context of AI refers to speculation about a new AI model being developed by OpenAI, codenamed "Strawberry." This model is rumored to have advanced capabilities beyond current AI models, potentially including planning, internet navigation, and "deep research." The hype began with a cryptic social media post by OpenAI CEO Sam Altman featuring strawberries.'}

In [83]:
agent_executor.invoke(
    {"input": "How to compile a text file to a pdf file?"},
    {"config": {"callbacks": [langfuse_handler]}}
)



> Entering new AgentExecutor chain...
Question: How to compile a text file to a pdf file?
Thought: I can use the retriever_tool to get information about compiling text files to pdf.
Action: retriever_tool
Action Input: How to compile a text file to a pdf file?Contents
1 Template 5
1.1 How tocompilea .texﬁleto a.pdfﬁle. . . . . . . . . . . . . 5
1.1.1 Tools . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5

Chapter 1
Template
1.1 How to compilea .texﬁleto a.pdfﬁle
1.1.1 Tools
To processthe ﬁlesyou(may)need:
•pdflatex (for example fromtetexpackage≥0.9-6, which you can
getfromRed Hat5.2 );

2.run“pdflatexfile.tex ”onthemainﬁleofthedocumentthreetimes
(three—to preparevalidtable ofcontents);
3.to see or print the result use acroread (unfortunately some versions of

1.3. LATEX ANDPDFL ATEXCAPABILITIES 7
1.3 LATEX andpdfLATEX capabilities
1.3.1 Overview
First you edit your source .texﬁle. In L ATEX you compile it using the latexThought: The provided text mentions using pdflatex to c

{'input': 'How to compile a text file to a pdf file?',
 'output': 'You can convert a text file to PDF using several methods:\n* **Online Converters:** Websites like Smallpdf and Zamzar offer free and easy conversion without software installation.\n* **Microsoft Word:** Open a new Word document, copy and paste your text, and save the file as a PDF. \n* **Other Software:**  Dedicated conversion tools offer more advanced features like batch processing and format customization.'}